In [90]:
import pandas as pd

#handle weird kaleido error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])

from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [91]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

s = requests.Session()
retries = Retry(total=25, backoff_factor=10, status_forcelist=[ 404, 500, 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2_Fees' in pwd:
    prepend = ''
else:
    prepend = 'L2_Fees/'

In [92]:
statuses = {x for x in range(100, 600)}
statuses.remove(200)
statuses.remove(429)

trailing_num_days = 120#90

start_date = date.today()-timedelta(days=trailing_num_days +1)

start_date = max(start_date, date(2021,11,12))
print(start_date)
# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()

2022-07-23


In [93]:
def get_cryptostats_api(api_core_string,dt_string ):
        dt_string = dt_string.strftime("%Y-%m-%d")
        i = 0
        res = pd.DataFrame()
        retry_client = RetryClient()
        # print(api_core_string + dt_string)
        good = 0
        n = 0
        while good == 1:
                with retry_client.get(api_core_string + dt_string, retry_options=ExponentialRetry(attempts=25), raise_for_status=statuses) as response:
                        try:
                                prot_req = response.json()
                                res = pd.json_normalize(prot_req['data']).reset_index()
                                res['date'] = dt_string
                        except Exception as e:
                                raise Exception("Could not convert json")
                res_aave = res[res['id'] == 'aave-v3-optimism-proto']
                res_aave_resp = res_aave['errors.oneDayFeeBreakdown']
                print(res_aave_resp)
                if (res_aave_resp == 'Execution timed out'):
                        good = 1
                        n = n+1
                else:
                        good = 0
                if n == 5:
                        good = 0
                retry_client.close()
        return res
        

In [94]:
# https://api.cryptostats.community/api/v1/<collection-id>/<query>/<arg>

api_core = 'https://api.cryptostats.community/api/v1/fees/oneDayFeeBreakdown/' #Can only pull everything all at once. Odd.
date_rng = pd.date_range(start=start_date, end=date.today()-timedelta(days=1))
date_rng_str = ','.join(x.strftime("%Y-%m-%d") for x in date_rng)

In [95]:
resp = s.get ('https://api.cryptostats.community/api/v1/fees/oneDayFeeBreakdown/2022-11-18')


In [96]:
# rpd = pd.json_normalize(resp.json()['data'])
rpd = resp.json()['data']['id' == 'aave-v3-optimism-proto']
print(rpd)
# print(rpd[rpd['metadata.name'] == 'Aave'])

{'id': 'compound', 'bundle': None, 'results': {'oneDayFeeBreakdown': None}, 'metadata': {'source': 'Messari Subgraph', 'icon': '', 'name': 'Compound', 'category': 'lending', 'description': 'Compound is an open borrowing & lending protocol.', 'feeDescription': 'Interest fees are paid from borrowers to lenders.', 'blockchain': 'Ethereum', 'website': 'https://compound.finance', 'tokenTicker': 'COMP', 'tokenCoingecko': 'compound-governance-token', 'protocolLaunch': '2019-05-07', 'tokenLaunch': '2020-06-22'}}


In [97]:
def get_range(date_range):
        data_dfs = []
        fee_df = []

        for dt in date_range:
                tdf = get_cryptostats_api(api_core,dt)
                data_dfs.append(tdf)

        
 
        # loop = asyncio.get_event_loop()
        # tasks = [get_cryptostats_api(api_core,dt) for dt in date_range]

        # data_dfs = loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))

        # data_dfs = [value for value in data_dfs if type(value) == pd.DataFrame]
        
        # # print(data_dfs)

        fee_df = pd.concat(data_dfs)
        return fee_df

In [98]:
fdf_raw = get_range(date_rng)

Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd1c9a71d60>
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd170e95ac0>
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd1c965a640>
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd170e95ac0>
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd1c965a640>
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd1c965a640>
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd1c9a71250>
Aiohttp retry client was not closed
Unclosed client session
client_session: 

In [99]:
fdf = fdf_raw.copy()
fdf = fdf[(fdf['metadata.blockchain'] == 'Optimism') & (fdf['metadata.name'] == 'Aave')]
display(fdf.columns)

KeyError: 'metadata.blockchain'

In [ ]:

# fdf = fdf[['date','metadata.name'
#                 ,'results.oneDayFeeBreakdown.treasuryIncomeUSD'
#                 ,'results.oneDayFeeBreakdown.outstandingTreasuryIncomeUSD'
#                 # ,'results.oneDayFeeBreakdown.lifetimeReserveFactorAccrued'
#         ]]

# fdf = fdf[~fdf['results.oneDayFeeBreakdown.treasuryIncomeUSD'].isna()]
display(fdf)
# fdf['results.oneDayFeeBreakdown.lifetimeReserveFactorAccrued'] = fdf['results.oneDayFeeBreakdown.lifetimeReserveFactorAccrued'].fillna(0)

# display(fdf)

KeyError: 'metadata.blockchain'